Traitements à faire
---
- Fusion de tous les Datasets
- Fusion des colonnes forename et surname en une colonne 
- Calculer l'age du pilote à la date de la course
- Retirer les colonnes inutilisées
- Exporter le Dataset fusionné

In [31]:
import sys
sys.path.append('../../')

import pandas as pd

In [32]:
from src.transformer_utils import print_df_missing_values

## 1. Chargement des Datasets

In [33]:
df_constructors = pd.read_csv('../../data/outputs/04_constructors_processed.csv')
df_drivers = pd.read_csv('../../data/outputs/03_drivers_processed.csv')
df_races_results = pd.read_csv('../../data/outputs/03_races_results.csv')
df_circuits = pd.read_csv('../../data/outputs/02_circuits_processed.csv')

In [34]:
df_constructors.head()

,constructorId,name,nationality,constructor_is_active
0,1,McLaren,British,1
1,2,BMW Sauber,German,0
2,3,Williams,British,1
3,4,Renault,French,0
4,5,Toro Rosso,Italian,0


In [35]:
df_drivers.head()

,driverId,number,forename,surname,dob,nationality,driver_is_active
0,1,44,Lewis,Hamilton,1985-01-07,British,1
1,2,\N,Nick,Heidfeld,1977-05-10,German,0
2,3,6,Nico,Rosberg,1985-06-27,German,0
3,4,14,Fernando,Alonso,1981-07-29,Spanish,1
4,5,\N,Heikki,Kovalainen,1981-10-19,Finnish,0


In [36]:
df_races_results.head()

,resultId,raceId,driverId,constructorId,grid,race_rank,points,laps,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,name
0,1,18,1,1,1,1,10.0,58,5690616,39,87452,218.300,1,2008,1,1,Australian Grand Prix
1,2,18,2,2,5,2,8.0,58,5696094,41,87739,217.586,1,2008,1,1,Australian Grand Prix
2,3,18,3,3,7,3,6.0,58,5698779,41,88090,216.719,1,2008,1,1,Australian Grand Prix
3,4,18,4,4,11,4,5.0,58,5707797,58,88603,215.464,1,2008,1,1,Australian Grand Prix
4,5,18,5,1,3,5,4.0,58,5708630,43,87418,218.385,1,2008,1,1,Australian Grand Prix


In [37]:
df_circuits.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url,circuits_is_active
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,1
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...,0
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...,1
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...,1
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park,0


## 2. Fusion des Datasets

### 2.1. Renommons les colonnes dans les Datasets

Etant donné que nous avons une colonne name dans le Dataset des resultats et des constructeurs, il est necessaire pour nous de renommer ces colonnes dans chacun des Dataset

In [38]:
df_races_results.rename({'name': 'circuit_name'}, axis=1, inplace=True)
df_constructors.rename({'name': 'constructor_name'}, axis=1, inplace=True)
df_constructors.rename({'nationality': 'country'}, axis=1, inplace=True)

### 2.2. Fusion des Datasets constructeurs et résultats

In [39]:
df_const_race = df_races_results.merge(df_constructors, on='constructorId', how='inner')

In [40]:
df_const_race.head()

,resultId,raceId,driverId,constructorId,grid,race_rank,points,laps,milliseconds,fastestLap,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,circuit_name,constructor_name,country,constructor_is_active
0,1,18,1,1,1,1,10.0,58,5690616,39,87452,218.300,1,2008,1,1,Australian Grand Prix,McLaren,British,1
1,2,18,2,2,5,2,8.0,58,5696094,41,87739,217.586,1,2008,1,1,Australian Grand Prix,BMW Sauber,German,0
2,3,18,3,3,7,3,6.0,58,5698779,41,88090,216.719,1,2008,1,1,Australian Grand Prix,Williams,British,1
3,4,18,4,4,11,4,5.0,58,5707797,58,88603,215.464,1,2008,1,1,Australian Grand Prix,Renault,French,0
4,5,18,5,1,3,5,4.0,58,5708630,43,87418,218.385,1,2008,1,1,Australian Grand Prix,McLaren,British,1


### 2.3. Fusion du Dataset des pilotes et des résultats des courses

In [41]:
# Visualisation du Dataset des pilotes
df_drivers.head()

,driverId,number,forename,surname,dob,nationality,driver_is_active
0,1,44,Lewis,Hamilton,1985-01-07,British,1
1,2,\N,Nick,Heidfeld,1977-05-10,German,0
2,3,6,Nico,Rosberg,1985-06-27,German,0
3,4,14,Fernando,Alonso,1981-07-29,Spanish,1
4,5,\N,Heikki,Kovalainen,1981-10-19,Finnish,0


#### a) Récupérons uniquement les colonnes utiles du Dataset des pilotes

In [42]:
cols = [x for x in df_drivers.columns if x not in ['number', 'code', 'nationality', 'url']]

In [43]:
df_drivers[cols].head()

,driverId,forename,surname,dob,driver_is_active
0,1,Lewis,Hamilton,1985-01-07,1
1,2,Nick,Heidfeld,1977-05-10,0
2,3,Nico,Rosberg,1985-06-27,0
3,4,Fernando,Alonso,1981-07-29,1
4,5,Heikki,Kovalainen,1981-10-19,0


#### b) Créons une colonne qui contient le nom et le prénom du pilote 

In [44]:
df_drivers['full_name'] = df_drivers['forename'] + ' ' + df_drivers['surname']
# Suppression des colonnes cars elles ne nous sont plus utiles
df_drivers.drop(['forename', 'surname', 'number'], axis=1, inplace=True)

#### c) Fusionnons tous les Datasets

In [45]:
df_merged = df_const_race.merge(df_drivers, on='driverId', how='inner')

In [46]:
print(df_merged.shape)
df_merged.head()

(26080, 24)


,resultId,raceId,driverId,constructorId,grid,race_rank,points,laps,milliseconds,fastestLap,...,round,circuitId,circuit_name,constructor_name,country,constructor_is_active,dob,nationality,driver_is_active,full_name
0,1,18,1,1,1,1,10.0,58,5690616,39,...,1,1,Australian Grand Prix,McLaren,British,1,1985-01-07,British,1,Lewis Hamilton
1,2,18,2,2,5,2,8.0,58,5696094,41,...,1,1,Australian Grand Prix,BMW Sauber,German,0,1977-05-10,German,0,Nick Heidfeld
2,3,18,3,3,7,3,6.0,58,5698779,41,...,1,1,Australian Grand Prix,Williams,British,1,1985-06-27,German,0,Nico Rosberg
3,4,18,4,4,11,4,5.0,58,5707797,58,...,1,1,Australian Grand Prix,Renault,French,0,1981-07-29,Spanish,1,Fernando Alonso
4,5,18,5,1,3,5,4.0,58,5708630,43,...,1,1,Australian Grand Prix,McLaren,British,1,1981-10-19,Finnish,0,Heikki Kovalainen


### 2.4. Fusion avec les circuits

In [47]:
df_merged = df_merged.merge(df_circuits[['circuitId', 'circuits_is_active']], on='circuitId', how='inner')

In [48]:
print(df_merged.shape)
df_merged.head()

(26080, 25)


,resultId,raceId,driverId,constructorId,grid,race_rank,points,laps,milliseconds,fastestLap,...,circuitId,circuit_name,constructor_name,country,constructor_is_active,dob,nationality,driver_is_active,full_name,circuits_is_active
0,1,18,1,1,1,1,10.0,58,5690616,39,...,1,Australian Grand Prix,McLaren,British,1,1985-01-07,British,1,Lewis Hamilton,1
1,2,18,2,2,5,2,8.0,58,5696094,41,...,1,Australian Grand Prix,BMW Sauber,German,0,1977-05-10,German,0,Nick Heidfeld,1
2,3,18,3,3,7,3,6.0,58,5698779,41,...,1,Australian Grand Prix,Williams,British,1,1985-06-27,German,0,Nico Rosberg,1
3,4,18,4,4,11,4,5.0,58,5707797,58,...,1,Australian Grand Prix,Renault,French,0,1981-07-29,Spanish,1,Fernando Alonso,1
4,5,18,5,1,3,5,4.0,58,5708630,43,...,1,Australian Grand Prix,McLaren,British,1,1981-10-19,Finnish,0,Heikki Kovalainen,1


## 4. Calcul de l'age de chaque pilote par course

In [49]:
def calcul_age(row: pd.DataFrame) -> int:
    return row['year'] - row['dob'].year

In [50]:
# Convertion de la colonne dob en type date Pandas
df_merged['dob'] = pd.to_datetime(df_merged['dob'])
df_merged['season_age'] = df_merged.apply(calcul_age, axis=1)

# Suppression de la colonne dob car elle ne sera plus utile pour la suite
df_merged.drop('dob', axis=1, inplace=True)

In [51]:
df_merged.head()

,resultId,raceId,driverId,constructorId,grid,race_rank,points,laps,milliseconds,fastestLap,...,circuitId,circuit_name,constructor_name,country,constructor_is_active,nationality,driver_is_active,full_name,circuits_is_active,season_age
0,1,18,1,1,1,1,10.0,58,5690616,39,...,1,Australian Grand Prix,McLaren,British,1,British,1,Lewis Hamilton,1,23
1,2,18,2,2,5,2,8.0,58,5696094,41,...,1,Australian Grand Prix,BMW Sauber,German,0,German,0,Nick Heidfeld,1,31
2,3,18,3,3,7,3,6.0,58,5698779,41,...,1,Australian Grand Prix,Williams,British,1,German,0,Nico Rosberg,1,23
3,4,18,4,4,11,4,5.0,58,5707797,58,...,1,Australian Grand Prix,Renault,French,0,Spanish,1,Fernando Alonso,1,27
4,5,18,5,1,3,5,4.0,58,5708630,43,...,1,Australian Grand Prix,McLaren,British,1,Finnish,0,Heikki Kovalainen,1,27


## 5. Verifions si on a aucune valeur manquante avant de poursuivre

In [52]:
car = '\\N'
print_df_missing_values(df_merged, car)

resultId: 0
raceId: 0
driverId: 0
constructorId: 0
grid: 0
race_rank: 0
points: 0
laps: 0
milliseconds: 0
fastestLap: 0
fastestLapTime: 0
fastestLapSpeed: 0
statusId: 0
year: 0
round: 0
circuitId: 0
circuit_name: 0
constructor_name: 0
country: 0
constructor_is_active: 0
nationality: 0
driver_is_active: 0
full_name: 0
circuits_is_active: 0
season_age: 0


## 5. Exportons le Dataset

In [53]:
df_merged.to_csv('../../data/outputs/05_merged_data.csv', index=False)